# Tryout

In [1]:
import torch
import torch.nn as nn

In [2]:
input_dim = 5
hidden_dim = 10
n_layers = 1

lstm_layer = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True)

In [3]:
# initialize hidden state of each cells
batch_size = 1
seq_len = 3

inp = torch.randn(batch_size, seq_len, input_dim)
hidden_state = torch.randn(n_layers, batch_size, hidden_dim)
cell_state = torch.randn(n_layers, batch_size, hidden_dim)
hidden = (hidden_state, cell_state)

print(inp.shape)
print(hidden_state.shape, cell_state.shape)

torch.Size([1, 3, 5])
torch.Size([1, 1, 10]) torch.Size([1, 1, 10])


In [4]:
# feed the input and hidden states and see what we’ll get back from it.
out, hidden = lstm_layer(inp, hidden)

print("Output shape: ", out.shape)
print("Hidden: ", hidden)

Output shape:  torch.Size([1, 3, 10])
Hidden:  (tensor([[[-8.4851e-02, -8.7748e-02,  6.2854e-02, -1.5089e-01,  7.9291e-05,
           1.6117e-01,  1.5947e-01,  1.4106e-01,  1.1534e-01,  1.3942e-01]]],
       grad_fn=<StackBackward>), tensor([[[-2.1368e-01, -1.8903e-01,  1.4674e-01, -3.4706e-01,  2.1826e-04,
           3.3946e-01,  3.7381e-01,  2.4257e-01,  3.2680e-01,  2.7158e-01]]],
       grad_fn=<StackBackward>))


In [5]:
# Obtaining the last output
out = out.squeeze()[-1, :]
print(out.shape)

torch.Size([10])


# Real implementation

In [7]:
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn

In [87]:
# preprocessing

# TotalFault_5_FlowCoolPressureDroppedBelowLimit-Flowcoolleak-FlowCoolPressureDroppedBelowLimit-FlowCoolPressureDroppedBelowLimit-FlowCoolPressureDroppedBelowLimit_Lot_runnum_9295-10815254

dir_path = "../phm_etching_01M01-02/M02_Groups_same_length/"
shuffle_indexes = [i for i in range(len(os.listdir(dir_path)))]
np.random.shuffle(shuffle_indexes)

data_x = np.zeros((len(os.listdir(dir_path)), 1005, 21))
data_y = pd.DataFrame({
    "Healthy": [0 for i in range(len(os.listdir(dir_path)))],
    "FlowCoolPressureDroppedBelowLimit": [0 for i in range(len(os.listdir(dir_path)))],
    "FlowcoolPressureTooHighCheckFlowcoolPump": [0 for i in range(len(os.listdir(dir_path)))],
    "Flowcoolleak": [0 for i in range(len(os.listdir(dir_path)))]
})

for fileIndex, file in enumerate(tqdm(os.listdir(dir_path))):
    
    fault_type_list = list(set(file.split("_")[2].split("-")))
    lot_runnum = file.split("_")[5].split(".")[0]
    
    data_import = pd.read_csv(dir_path + file, encoding="utf8")
    
    data_import.drop(columns=['time', 'Tool', 'Lot', 'TTF_FlowCool Pressure Dropped Below Limit',
       'TTF_Flowcool Pressure Too High Check Flowcool Pump', 'TTF_Flowcool leak', 'Lot-runnum'],
                     inplace=True)
    
    data_x[shuffle_indexes[fileIndex]] = data_import.to_numpy()
    
    for faultType in ["Healthy", "FlowCoolPressureDroppedBelowLimit",
                      "FlowcoolPressureTooHighCheckFlowcoolPump", "Flowcoolleak"]:
        if faultType in fault_type_list:
            data_y.loc[shuffle_indexes[fileIndex], faultType] = 1
        
data_y = data_y.to_numpy()
        
print(data_x.shape)
print(data_y.shape)

100%|██████████| 5013/5013 [00:38<00:00, 129.37it/s]

(5013, 1005, 21)
(5013, 4)


In [92]:
# split for training, validation, testing
training_ratio = 0.7
validation_ratio = 0.2
testing_ratio = 0.1

train_indice = int(np.around(len(data_x) * training_ratio, 0))
validation_indice = int(np.around(len(data_x) * training_ratio, 0)) +\
int(np.around(len(data_x) * validation_ratio, 0))

x_train = data_x[:train_indice]
y_train = data_y[:train_indice]
x_validation = data_x[train_indice:validation_indice]
y_validation = data_y[train_indice:validation_indice]
x_test = data_x[validation_indice:]
y_test = data_y[train_indice:validation_indice]

In [94]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()
print(is_cuda)

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

True
